In [1]:
# read in any libraries i will need
# Import libraries i will need
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

from osgeo import gdal
import random as python_random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils.vis_utils import plot_model
import gc
import keras

#import the libraries necessary for confusion plot
from sklearn.metrics import confusion_matrix
import seaborn as sns
import cv2 # imports the computer vision package
from tensorflow.keras.models import Model
from matplotlib import colors

In [2]:
inputPath="D:/final_data/2017_2"
filelist = []

# Load the images, and append them to a list.
for filepath in os.listdir(inputPath):
    if filepath.endswith((".tif")):
    #print(filepath)
        tempfile=inputPath+'/{0}'.format(filepath)
        filelist.append(tempfile)
    
#### 7/25/22 #######

# Switching this up, 15= depth, 16=tree/no tree, 16 = NDVI... want to switch NDVI to not vegetation <0

In [3]:
from keras import backend as K

def jaccard_distance_loss(y_true, y_pred, smooth=100):
    """
    Jaccard = (|X & Y|)/ (|X|+ |Y| - |X & Y|)
            = sum(|A*B|)/(sum(|A|)+sum(|B|)-sum(|A*B|))
    
    The jaccard distance loss is usefull for unbalanced datasets. This has been
    shifted so it converges on 0 and is smoothed to avoid exploding or disapearing
    gradient.
    
    Ref: https://en.wikipedia.org/wiki/Jaccard_index
    
    @url: https://gist.github.com/wassname/f1452b748efcbeb4cb9b1d059dce6f96
    @author: wassname
    """
    intersection = K.sum(K.sum(K.abs(y_true * y_pred), axis=-1))
    sum_ = K.sum(K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1))
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return (1 - jac) * smooth

def dice_metric(y_pred, y_true):
    intersection = K.sum(K.sum(K.abs(y_true * y_pred), axis=-1))
    union = K.sum(K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1))
    # if y_pred.sum() == 0 and y_pred.sum() == 0:
    #     return 1.0

    return 2*intersection / union

# define my own jaccard metric, should be same as IoU
def jaccard_coef(y_true, y_pred):
    y_true_f=K.flatten(y_true)
    y_pred_f=K.flatten(y_pred)
    intersection=K.sum(y_true_f + y_pred_f)
    return (intersection +1.0)/ (K.sum(y_true_f) + K.sum(y_pred_f) - intersection +1.0)
    
def jaccard_coef_loss(y_true, y_pred):
    return -jaccard_coef(y_true, y_pred)

In [4]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, batch_size=25, shuffle=True):
        'Initialization'
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, Y1, Y2, Y3 = self.__data_generation(list_IDs_temp)
        y=[Y1,Y2,Y3]

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)


    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        images = []

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            dataset = gdal.Open(ID)
            image = dataset.ReadAsArray()  # Returned image is a NumPy array with shape (16, 60, 60) for example.
            images.append(image)  # Append the NumPy array to the list.

        all_data= np.stack(images, axis= 0)
        all_data[all_data < .0000001] = 0
        X=all_data[:,:14,:,:] # separate out the band values
        X = np.transpose(X, axes=[0, 2, 3, 1])
        # normalize values of the input data to 0,1
        X = X/X.max(axis=(3),keepdims=1)
        # For RGB uncomment this
        X = X[:,:,:,:3]
        
        # canopy_height,tree/not tree,ndvi
        all_data= np.stack( images, axis= 0)
        Y = all_data[:,14:]
        Y[Y  < .0000001] = 0
        #Y[:,0][Y[:,0]  > 1] = 1
        Y1=Y[:,0] # 0 for height, 1 for tree/not, 2 for NDVI 
        Y1 = Y1/Y1.max()          
        Y2=Y[:,1]
        Y2[Y2  >0 ] = 1 
        all_data= np.stack( images, axis= 0)
        Y3 = all_data[:,14:]
        Y3[(Y3 < .0000001) & (Y3 >= 0) ] = 1
        Y3=Y3[:,2] # 0 for height, 1 for tree/not, 2 for NDVI 
        Y3[Y3  >0 ] = 0
        Y3[Y3  <0 ] = 1
        
        
        #Y[:,0][Y[:,0]  > 1] = 1
        #Y3=Y[:,2] # 0 for height, 1 for tree/not, 2 for NDVI 
        #Y3 = Y3/Y3.max()
        self.X=X
        self.Y1=Y1
        self.Y2=Y2
        self.Y3=Y3

        return X, Y1, Y2, Y3
    
    
    def get_true_values_x(self, indexes):
        'Generate one batch of data'
        # Return validation data for plotting 

        # Find list of IDs - (give input of validation indexes)
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, Y1, Y2, Y3 = self.__data_generation(list_IDs_temp)
        y=[Y1,Y2,Y3]
        
        # only need to return the RGB data for plotting
        X = X[:,:,:,0:3]

        return X
    
    def get_true_values_y(self, indexes):
        'Generate one batch of data'
        # Return validation data for plotting 

        # Find list of IDs - (give input of validation indexes)
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, Y1, Y2, Y3 = self.__data_generation(list_IDs_temp)
        y=[Y1,Y2,Y3]

        return y

In [5]:
params = {'batch_size':12,
         'shuffle': True}

# need a dictionary which has a list of training paths and a list of validations paths
filelist_temp = filelist
np.random.seed(14)
mask = np.random.rand(len(filelist_temp)) <=.75

training_data = np.array(filelist_temp)[mask]
val_data = np.array(filelist_temp)[~mask]


mydict = {}
mydict["training"] = training_data
mydict["validation"] = val_data

# generators
training_generator = DataGenerator(mydict["training"], **params)
val_generator = DataGenerator(mydict["validation"], **params)

len(filelist_temp)

9534

In [6]:
# =====================================================
# define U-Net model architecture - All shared layers

def build_unet(img_shape):
    # input layer shape is equal to patch image size
    inputs = layers.Input(shape=img_shape)

    # rescale images from (0, 255) to (0, 1)
 #   rescale = Rescaling(scale=1. / 255, input_shape=(img_height, img_width, img_channels))(inputs)
 #   previous_block_activation = rescale  # Set aside residual
    previous_block_activation = inputs

    contraction = {}
    # # Contraction path: Blocks 1 through 5 are identical apart from the feature depth
    for f in [32, 64]:
        x = layers.Conv2D(f, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(previous_block_activation)
        x = layers.Dropout(0.1)(x)
        x = layers.Conv2D(f, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
        contraction[f'conv{f}'] = x
        x = layers.MaxPooling2D((2, 2))(x)
        previous_block_activation = x

    x = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(previous_block_activation)
    x = layers.Dropout(0.1)(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
    contraction[f'conv{128}'] = x
    x = layers.MaxPooling2D((3, 3))(x)
    previous_block_activation = x
        
    c5 = layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(previous_block_activation)
    c5 = layers.Dropout(0.2)(c5)
    c5 = layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)
    previous_block_activation = c5
    
    x = layers.Conv2DTranspose(128, (2, 2), strides=(3, 3), padding='same')(previous_block_activation)
    x = layers.concatenate([x, contraction[f'conv{128}']])
    x = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
    previous_block_activation = x
        
    # Expansive path: Second half of the network: upsampling inputs
    # could we use upsampling layers here instead of Conv2dTRanspose layers? might that help
    for f in reversed([32, 64]):
        x = layers.Conv2DTranspose(f, (2, 2), strides=(2, 2), padding='same')(previous_block_activation)
        x = layers.concatenate([x, contraction[f'conv{f}']])
        x = layers.Conv2D(f, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
        x = layers.Dropout(0.2)(x)
        x = layers.Conv2D(f, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
        previous_block_activation = x

    output_tree_height = layers.Conv2D(filters=1, kernel_size=(1, 1), activation="linear", name="tree_height")(previous_block_activation)

    output_tree_binary = layers.Conv2D(filters=1, kernel_size=(1, 1), activation="sigmoid", name="tree_binary")(previous_block_activation)
    
    output_vegetation_task = layers.Conv2D(filters=1, kernel_size=(1, 1), activation="sigmoid", name="vegetation_task")(previous_block_activation)


    return Model(inputs=inputs, outputs=[output_tree_height,output_tree_binary,output_vegetation_task])

In [7]:
#plot_model(model_multi_all_shared,"multi_task_model_all_shared_final.png" , show_shapes=True, show_layer_names=True)

# build model
model_multi_all_shared= build_unet(img_shape=(240, 240, 3))
model_multi_all_shared.summary()

model_multi_all_shared.compile(optimizer="adam", 
              loss={'tree_height': 'mse',
                    'tree_binary': jaccard_distance_loss,
                    'vegetation_task': jaccard_distance_loss,},
               loss_weights={'tree_binary': .5, 
                             'tree_height': .6,
                             'vegetation_task': .1,},
              metrics={'tree_height': ["mae", 'accuracy'],
                       'tree_binary': [tf.keras.metrics.BinaryCrossentropy(),tf.keras.metrics.MeanIoU(num_classes=2),dice_metric], 
                       'vegetation_task': [tf.keras.metrics.BinaryCrossentropy(),tf.keras.metrics.MeanIoU(num_classes=2),dice_metric]}
)
#print(model_multi_all_shared.metrics_names)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 240, 240, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 240, 240, 32  896         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 dropout (Dropout)              (None, 240, 240, 32  0           ['conv2d[0][0]']                 
                                )                                                             

                                )                                                                 
                                                                                                  
 tree_height (Conv2D)           (None, 240, 240, 1)  33          ['conv2d_13[0][0]']              
                                                                                                  
 tree_binary (Conv2D)           (None, 240, 240, 1)  33          ['conv2d_13[0][0]']              
                                                                                                  
 vegetation_task (Conv2D)       (None, 240, 240, 1)  33          ['conv2d_13[0][0]']              
                                                                                                  
Total params: 1,925,667
Trainable params: 1,925,667
Non-trainable params: 0
__________________________________________________________________________________________________


In [8]:
%%time

callback = tf.keras.callbacks.EarlyStopping(monitor="val_tree_binary_mean_io_u", patience=5, restore_best_weights=True,
                                           mode="max", verbose=1)

# Train our model
history=model_multi_all_shared.fit(
    training_generator,
    epochs=100,
    validation_data=val_generator,
    callbacks=callback
)

# model_multi_all_shared.save('C:/Users/johnf/Documents/UCL/thesis/code/models/multi_all_shared')

# hist_df = pd.DataFrame(history.history)

# hist_df
# hist_df.to_csv('C:/Users/johnf/Documents/UCL/thesis/code/models/multi_all_shared_hist.csv')  

# hist_df

Epoch 1/100
597/597 [==============================] - 1446s 2s/step - loss: 40.8171 - tree_height_loss: 0.1551 - tree_binary_loss: 78.7111 - vegetation_task_loss: 13.6848 - tree_height_mae: 0.2434 - tree_height_accuracy: 0.0718 - tree_binary_binary_crossentropy: 0.7414 - tree_binary_mean_io_u: 0.5251 - tree_binary_dice_metric: 0.3459 - vegetation_task_binary_crossentropy: 0.5529 - vegetation_task_mean_io_u_1: 0.1535 - vegetation_task_dice_metric: 0.9251 - val_loss: 37.7890 - val_tree_height_loss: 0.0161 - val_tree_binary_loss: 73.6702 - val_vegetation_task_loss: 9.4429 - val_tree_height_mae: 0.0887 - val_tree_height_accuracy: 0.0767 - val_tree_binary_binary_crossentropy: 0.9054 - val_tree_binary_mean_io_u: 0.5801 - val_tree_binary_dice_metric: 0.4153 - val_vegetation_task_binary_crossentropy: 0.4752 - val_vegetation_task_mean_io_u_1: 0.1592 - val_vegetation_task_dice_metric: 0.9503
Epoch 2/100
597/597 [==============================] - 852s 1s/step - loss: 37.6770 - tree_height_loss: 

597/597 [==============================] - 955s 2s/step - loss: 34.0339 - tree_height_loss: 0.0160 - tree_binary_loss: 66.7293 - vegetation_task_loss: 6.5964 - tree_height_mae: 0.0887 - tree_height_accuracy: 0.0765 - tree_binary_binary_crossentropy: 0.8368 - tree_binary_mean_io_u: 0.6349 - tree_binary_dice_metric: 0.4973 - vegetation_task_binary_crossentropy: 0.6003 - vegetation_task_mean_io_u_1: 0.8241 - vegetation_task_dice_metric: 0.9658 - val_loss: 33.8434 - val_tree_height_loss: 0.0152 - val_tree_binary_loss: 66.4545 - val_vegetation_task_loss: 6.0712 - val_tree_height_mae: 0.0971 - val_tree_height_accuracy: 0.0768 - val_tree_binary_binary_crossentropy: 0.8183 - val_tree_binary_mean_io_u: 0.6357 - val_tree_binary_dice_metric: 0.5006 - val_vegetation_task_binary_crossentropy: 0.5428 - val_vegetation_task_mean_io_u_1: 0.8499 - val_vegetation_task_dice_metric: 0.9686
Epoch 20/100
597/597 [==============================] - 935s 2s/step - loss: 33.8669 - tree_height_loss: 0.0184 - tree

597/597 [==============================] - 970s 2s/step - loss: 33.4168 - tree_height_loss: 0.0176 - tree_binary_loss: 65.5585 - vegetation_task_loss: 6.2704 - tree_height_mae: 0.0929 - tree_height_accuracy: 0.0764 - tree_binary_binary_crossentropy: 0.8395 - tree_binary_mean_io_u: 0.6423 - tree_binary_dice_metric: 0.5105 - vegetation_task_binary_crossentropy: 0.6397 - vegetation_task_mean_io_u_1: 0.8712 - vegetation_task_dice_metric: 0.9676 - val_loss: 33.4470 - val_tree_height_loss: 0.0219 - val_tree_binary_loss: 65.6734 - val_vegetation_task_loss: 5.9715 - val_tree_height_mae: 0.1040 - val_tree_height_accuracy: 0.0768 - val_tree_binary_binary_crossentropy: 0.8057 - val_tree_binary_mean_io_u: 0.6412 - val_tree_binary_dice_metric: 0.5093 - val_vegetation_task_binary_crossentropy: 0.6212 - val_vegetation_task_mean_io_u_1: 0.8873 - val_vegetation_task_dice_metric: 0.9692
Epoch 38/100
597/597 [==============================] - 941s 2s/step - loss: 33.4397 - tree_height_loss: 0.0188 - tree

KeyboardInterrupt: 

In [5]:
# print(tf.reduce_sum(tf.random.normal([1000, 1000])))
# print(tf.config.list_physical_devices('GPU'))
# len(tf.config.list_physical_devices('GPU')) > 0 



params = {'batch_size':8,
         'shuffle': True}

# need a dictionary which has a list of training paths and a list of validations paths
filelist_temp = filelist
np.random.seed(14)
mask = np.random.rand(len(filelist_temp)) <=.75

training_data = np.array(filelist_temp)[mask]
val_data = np.array(filelist_temp)[~mask]


mydict = {}
mydict["training"] = training_data
mydict["validation"] = val_data

# generators
training_generator = DataGenerator(mydict["training"], **params)
val_generator = DataGenerator(mydict["validation"], **params)

In [6]:
# =====================================================
# define U-Net model architecture - proof_concept_2
def build_unet(img_shape):
    # input layer shape is equal to patch image size
    inputs = layers.Input(shape=img_shape)

    # rescale images from (0, 255) to (0, 1)
 #   rescale = Rescaling(scale=1. / 255, input_shape=(img_height, img_width, img_channels))(inputs)
 #   previous_block_activation = rescale  # Set aside residual
    previous_block_activation = inputs

    contraction = {}
    # # Contraction path: Blocks 1 through 5 are identical apart from the feature depth
    for f in [32, 64]:
        x = layers.Conv2D(f, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(previous_block_activation)
        x = layers.Dropout(0.1)(x)
        x = layers.Conv2D(f, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
        contraction[f'conv{f}'] = x
        x = layers.MaxPooling2D((2, 2))(x)
        previous_block_activation = x

    x = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(previous_block_activation)
    x = layers.Dropout(0.1)(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
    contraction[f'conv{128}'] = x
    x = layers.MaxPooling2D((3, 3))(x)
    previous_block_activation = x
        
    c5 = layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(previous_block_activation)
    c5 = layers.Dropout(0.2)(c5)
    c5 = layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)
    previous_block_activation0 = c5
    
    x = layers.Conv2DTranspose(128, (2, 2), strides=(3, 3), padding='same')(previous_block_activation0)
    x = layers.concatenate([x, contraction[f'conv{128}']])
    x = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
    previous_block_activation_1 = x
    
    x = layers.Conv2DTranspose(128, (2, 2), strides=(3, 3), padding='same')(previous_block_activation0)
    x = layers.concatenate([x, contraction[f'conv{128}']])
    x = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
    previous_block_activation_2 = x
    
    x = layers.Conv2DTranspose(128, (2, 2), strides=(3, 3), padding='same')(previous_block_activation0)
    x = layers.concatenate([x, contraction[f'conv{128}']])
    x = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
    previous_block_activation_3 = x  

    # Expansive path: Second half of the network: upsampling inputs
    # could we use upsampling layers here instead of Conv2dTRanspose layers? might that help
    for f in reversed([32, 64]):
        x = layers.Conv2DTranspose(f, (2, 2), strides=(2, 2), padding='same')(previous_block_activation_1)
        x = layers.concatenate([x, contraction[f'conv{f}']])
        x = layers.Conv2D(f, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
        x = layers.Dropout(0.2)(x)
        x = layers.Conv2D(f, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
        previous_block_activation_1 = x

            # Expansive path: Second half of the network: upsampling inputs
    # could we use upsampling layers here instead of Conv2dTRanspose layers? might that help
    for f in reversed([32, 64]):
        x = layers.Conv2DTranspose(f, (2, 2), strides=(2, 2), padding='same')(previous_block_activation_2)
        x = layers.concatenate([x, contraction[f'conv{f}']])
        x = layers.Conv2D(f, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
        x = layers.Dropout(0.2)(x)
        x = layers.Conv2D(f, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
        previous_block_activation_2 = x

            # Expansive path: Second half of the network: upsampling inputs
    # could we use upsampling layers here instead of Conv2dTRanspose layers? might that help
    for f in reversed([32, 64]):
        x = layers.Conv2DTranspose(f, (2, 2), strides=(2, 2), padding='same')(previous_block_activation_3)
        x = layers.concatenate([x, contraction[f'conv{f}']])
        x = layers.Conv2D(f, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
        x = layers.Dropout(0.2)(x)
        x = layers.Conv2D(f, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
        previous_block_activation_3 = x
        
    output_tree_height = layers.Conv2D(filters=1, kernel_size=(1, 1), activation="linear", name="tree_height")(previous_block_activation_1)
    output_tree_binary = layers.Conv2D(filters=1, kernel_size=(1, 1), activation="sigmoid", name="tree_binary")(previous_block_activation_2)    
    output_vegetation_task = layers.Conv2D(filters=1, kernel_size=(1, 1), activation="sigmoid", name="vegetation_task")(previous_block_activation_3)


    return Model(inputs=inputs, outputs=[output_tree_height,output_tree_binary,output_vegetation_task])

In [7]:
# build model
model_multi_partial_shared= build_unet(img_shape=(240, 240, 3))
model_multi_partial_shared.summary()
# plot_model(model_unet_multitask_noweights,"multi_task_model_fuller_attempt.png" , show_shapes=True, show_layer_names=True)
model_multi_partial_shared.compile(optimizer="adam", 
              loss={'tree_height': 'mse',
                    'tree_binary': jaccard_distance_loss,
                    'vegetation_task': jaccard_distance_loss,},
               loss_weights={'tree_binary': .6, 
                             'tree_height': .6,
                             'vegetation_task': .1,},
              metrics={'tree_height': ["mae", 'accuracy'],
                       'tree_binary': [tf.keras.metrics.BinaryCrossentropy(),tf.keras.metrics.MeanIoU(num_classes=2),dice_metric], 
                       'vegetation_task': [tf.keras.metrics.BinaryCrossentropy(),tf.keras.metrics.MeanIoU(num_classes=2),dice_metric]}
)
#print(model_multi_all_shared.metrics_names)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 240, 240, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 240, 240, 32  896         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 dropout (Dropout)              (None, 240, 240, 32  0           ['conv2d[0][0]']                 
                                )                                                             

 spose)                         )                                                                 
                                                                                                  
 conv2d_transpose_7 (Conv2DTran  (None, 120, 120, 64  32832      ['conv2d_13[0][0]']              
 spose)                         )                                                                 
                                                                                                  
 concatenate_3 (Concatenate)    (None, 120, 120, 12  0           ['conv2d_transpose_3[0][0]',     
                                8)                                'conv2d_3[0][0]']               
                                                                                                  
 concatenate_5 (Concatenate)    (None, 120, 120, 12  0           ['conv2d_transpose_5[0][0]',     
                                8)                                'conv2d_3[0][0]']               
          

 conv2d_25 (Conv2D)             (None, 240, 240, 32  9248        ['dropout_12[0][0]']             
                                )                                                                 
                                                                                                  
 tree_height (Conv2D)           (None, 240, 240, 1)  33          ['conv2d_17[0][0]']              
                                                                                                  
 tree_binary (Conv2D)           (None, 240, 240, 1)  33          ['conv2d_21[0][0]']              
                                                                                                  
 vegetation_task (Conv2D)       (None, 240, 240, 1)  33          ['conv2d_25[0][0]']              
                                                                                                  
Total params: 3,432,291
Trainable params: 3,432,291
Non-trainable params: 0
_________________________________

In [8]:
%%time

callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1)

# Train our model
history=model_multi_partial_shared.fit(
    training_generator,
    epochs=100,
    validation_data=val_generator,
    callbacks=callback
)

# model_multi_partial_shared.save('C:/Users/johnf/Documents/UCL/thesis/code/models/multi_partial_shared')

# hist_df = pd.DataFrame(history.history)

# hist_df
# hist_df.to_csv('C:/Users/johnf/Documents/UCL/thesis/code/models/multi_partial_shared_hist.csv')  

# need to do three separate models with all of the data to compare against...

Epoch 1/100
896/896 [==============================] - 868s 960ms/step - loss: 59.5797 - tree_height_loss: 0.0189 - tree_binary_loss: 95.1461 - vegetation_task_loss: 24.8071 - tree_height_mae: 0.1009 - tree_height_accuracy: 0.0765 - tree_binary_binary_crossentropy: 14.0768 - tree_binary_mean_io_u: 0.0400 - tree_binary_dice_metric: 0.0920 - vegetation_task_binary_crossentropy: 3.7666 - vegetation_task_mean_io_u_1: 0.3766 - vegetation_task_dice_metric: 0.8570 - val_loss: 59.5899 - val_tree_height_loss: 0.0186 - val_tree_binary_loss: 95.1574 - val_vegetation_task_loss: 24.8439 - val_tree_height_mae: 0.0974 - val_tree_height_accuracy: 0.0768 - val_tree_binary_binary_crossentropy: 14.5139 - val_tree_binary_mean_io_u: 0.0241 - val_tree_binary_dice_metric: 0.0918 - val_vegetation_task_binary_crossentropy: 3.7893 - val_vegetation_task_mean_io_u_1: 0.3758 - val_vegetation_task_dice_metric: 0.8567
Epoch 2/100
896/896 [==============================] - 911s 1s/step - loss: 49.2869 - tree_height_l

896/896 [==============================] - 925s 1s/step - loss: 41.7348 - tree_height_loss: 0.0118 - tree_binary_loss: 68.4356 - vegetation_task_loss: 6.6635 - tree_height_mae: 0.0712 - tree_height_accuracy: 0.0765 - tree_binary_binary_crossentropy: 0.9568 - tree_binary_mean_io_u: 0.6237 - tree_binary_dice_metric: 0.4769 - vegetation_task_binary_crossentropy: 0.7024 - vegetation_task_mean_io_u_1: 0.8711 - vegetation_task_dice_metric: 0.9654 - val_loss: 41.6431 - val_tree_height_loss: 0.0122 - val_tree_binary_loss: 68.2829 - val_vegetation_task_loss: 6.6608 - val_tree_height_mae: 0.0696 - val_tree_height_accuracy: 0.0768 - val_tree_binary_binary_crossentropy: 0.9947 - val_tree_binary_mean_io_u: 0.6233 - val_tree_binary_dice_metric: 0.4791 - val_vegetation_task_binary_crossentropy: 0.7073 - val_vegetation_task_mean_io_u_1: 0.8780 - val_vegetation_task_dice_metric: 0.9655
Epoch 20/100
896/896 [==============================] - 932s 1s/step - loss: 41.7497 - tree_height_loss: 0.0117 - tree

###### tree yes/no #######

In [9]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, batch_size=25, shuffle=True):
        'Initialization'
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)


    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        images = []

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            dataset = gdal.Open(ID)
            image = dataset.ReadAsArray()  # Returned image is a NumPy array with shape (16, 60, 60) for example.
            images.append(image)  # Append the NumPy array to the list.

        all_data= np.stack(images, axis= 0)
        all_data[all_data < .0000001] = 0
        X=all_data[:,:14,:,:] # separate out the band values
        X = np.transpose(X, axes=[0, 2, 3, 1])
        # normalize values of the input data to 0,1
        X = X/X.max(axis=(3),keepdims=1)
        # For RGB uncomment this
        X = X[:,:,:,:3]
        
        # canopy_height,tree/not tree,ndvi
        all_data= np.stack( images, axis= 0)
        Y = all_data[:,14:]
        Y[Y  < .0000001] = 0
        Y[:,1][Y[:,1]  > 1] = 1
        Y=Y[:,1]
        self.X=X
        self.Y=Y

        return X, Y
    
    def get_true_values_x(self, indexes):
        'Generate one batch of data'
        # Return validation data for plotting 

        # Find list of IDs - (give input of validation indexes)
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)
        
        # only need to return the RGB data for plotting
        X = X[:,:,:,0:3]

        return X
    
    def get_true_values_y(self, indexes):
        'Generate one batch of data'
        # Return validation data for plotting 

        # Find list of IDs - (give input of validation indexes)
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return y

In [10]:
params = {'batch_size':12,
         'shuffle': True}

# need a dictionary which has a list of training paths and a list of validations paths
filelist_temp = filelist
np.random.seed(14)
mask = np.random.rand(len(filelist_temp)) <=.75

training_data = np.array(filelist_temp)[mask]
val_data = np.array(filelist_temp)[~mask]


mydict = {}
mydict["training"] = training_data
mydict["validation"] = val_data

# generators
training_generator = DataGenerator(mydict["training"], **params)
val_generator = DataGenerator(mydict["validation"], **params)

In [11]:
# =====================================================
# define U-Net model architecture - proof_concept_2

def build_unet(img_shape):
    # input layer shape is equal to patch image size
    inputs = layers.Input(shape=img_shape)

    # rescale images from (0, 255) to (0, 1)
 #   rescale = Rescaling(scale=1. / 255, input_shape=(img_height, img_width, img_channels))(inputs)
 #   previous_block_activation = rescale  # Set aside residual
    previous_block_activation = inputs

    contraction = {}
    # # Contraction path: Blocks 1 through 5 are identical apart from the feature depth
    for f in [32, 64]:
        x = layers.Conv2D(f, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(previous_block_activation)
        x = layers.Dropout(0.1)(x)
        x = layers.Conv2D(f, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
        contraction[f'conv{f}'] = x
        x = layers.MaxPooling2D((2, 2))(x)
        previous_block_activation = x

    x = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(previous_block_activation)
    x = layers.Dropout(0.1)(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
    contraction[f'conv{128}'] = x
    x = layers.MaxPooling2D((3, 3))(x)
    previous_block_activation = x
        
    c5 = layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(previous_block_activation)
    c5 = layers.Dropout(0.2)(c5)
    c5 = layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)
    previous_block_activation = c5
    
    x = layers.Conv2DTranspose(128, (2, 2), strides=(3, 3), padding='same')(previous_block_activation)
    x = layers.concatenate([x, contraction[f'conv{128}']])
    x = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
    previous_block_activation = x
        
    # Expansive path: Second half of the network: upsampling inputs
    # could we use upsampling layers here instead of Conv2dTRanspose layers? might that help
    for f in reversed([32, 64]):
        x = layers.Conv2DTranspose(f, (2, 2), strides=(2, 2), padding='same')(previous_block_activation)
        x = layers.concatenate([x, contraction[f'conv{f}']])
        x = layers.Conv2D(f, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
        x = layers.Dropout(0.2)(x)
        x = layers.Conv2D(f, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
        previous_block_activation = x

    outputs = layers.Conv2D(filters=1, kernel_size=(1, 1), activation="sigmoid")(previous_block_activation)

    return Model(inputs=inputs, outputs=outputs)

In [12]:
%%time
# build model
model_unet_tree_footprint_all = build_unet(img_shape=(240, 240, 3))
model_unet_tree_footprint_all.summary()

# compile model
model_unet_tree_footprint_all.compile(optimizer="adam",
              loss="binary_crossentropy", 
              metrics=[tf.keras.metrics.BinaryCrossentropy(),'accuracy',tf.keras.metrics.MeanIoU(num_classes=2)])

callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

# Train our model
history=model_unet_tree_footprint_all.fit(
    training_generator,
    epochs=100,
    validation_data=val_generator,
    callbacks=callback
)


# model_unet_tree_footprint_all.save('C:/Users/johnf/Documents/UCL/thesis/code/models/tree_model')

# hist_df = pd.DataFrame(history.history)

# hist_df
#hist_df.to_csv('C:/Users/johnf/Documents/UCL/thesis/code/models/tree_binary_hist.csv')  

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 240, 240, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_26 (Conv2D)             (None, 240, 240, 32  896         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 dropout_13 (Dropout)           (None, 240, 240, 32  0           ['conv2d_26[0][0]']              
                                )                                                           

                                )                                                                 
                                                                                                  
 conv2d_40 (Conv2D)             (None, 240, 240, 1)  33          ['conv2d_39[0][0]']              
                                                                                                  
Total params: 1,925,601
Trainable params: 1,925,601
Non-trainable params: 0
__________________________________________________________________________________________________
Epoch 1/100
597/597 [==============================] - 921s 2s/step - loss: 0.1396 - binary_crossentropy: 0.1396 - accuracy: 0.9518 - mean_io_u_2: 0.4759 - val_loss: 0.1232 - val_binary_crossentropy: 0.1232 - val_accuracy: 0.9518 - val_mean_io_u_2: 0.4759
Epoch 2/100
597/597 [==============================] - 916s 2s/step - loss: 0.1224 - binary_crossentropy: 0.1224 - accuracy: 0.9521 - mean_io_u_2: 0.4759 - val_loss: 0.1196 

In [13]:
## Canopy Height


class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, batch_size=25, shuffle=True):
        'Initialization'
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)


    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        images = []

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            dataset = gdal.Open(ID)
            image = dataset.ReadAsArray()  # Returned image is a NumPy array with shape (16, 60, 60) for example.
            images.append(image)  # Append the NumPy array to the list.

        all_data= np.stack(images, axis= 0)
        all_data[all_data < .0000001] = 0
        X=all_data[:,:14,:,:] # separate out the band values
        X = np.transpose(X, axes=[0, 2, 3, 1])
        # normalize values of the input data to 0,1
        X = X/X.max(axis=(3),keepdims=1)
        # For RGB uncomment this
        X = X[:,:,:,:3]
        
        # canopy_height,tree/not tree,ndvi
        all_data= np.stack( images, axis= 0)
        Y = all_data[:,14:]
        Y[Y  < .0000001] = 0
        #Y[:,0][Y[:,0]  > 1] = 1
        Y=Y[:,0]
        Y = Y/Y.max()
        self.X=X
        self.Y=Y

        return X, Y
    
    def get_true_values_x(self, indexes):
        'Generate one batch of data'
        # Return validation data for plotting 

        # Find list of IDs - (give input of validation indexes)
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        images = []

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            dataset = gdal.Open(ID)
            image = dataset.ReadAsArray()  # Returned image is a NumPy array with shape (16, 60, 60) for example.
            images.append(image)  # Append the NumPy array to the list.

        all_data= np.stack(images, axis= 0)
        all_data[all_data < .0000001] = 0
        X=all_data[:,:14,:,:] # separate out the band values
        X = np.transpose(X, axes=[0, 2, 3, 1])
        
        # only need to return the RGB data for plotting
        X = X[:,:,:,0:3]

        return X
    
    def get_true_values_y(self, indexes):
        'Generate one batch of data'
        # Return validation data for plotting 

        # Find list of IDs - (give input of validation indexes)
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, Y1 = self.__data_generation(list_IDs_temp)
        y=[Y1]

        return y

In [14]:
params = {'batch_size':15,
         'shuffle': True}

# need a dictionary which has a list of training paths and a list of validations paths
filelist_temp = filelist
np.random.seed(14)
mask = np.random.rand(len(filelist_temp)) <=.75

training_data = np.array(filelist_temp)[mask]
val_data = np.array(filelist_temp)[~mask]


mydict = {}
mydict["training"] = training_data
mydict["validation"] = val_data

# generators
training_generator = DataGenerator(mydict["training"], **params)
val_generator = DataGenerator(mydict["validation"], **params)

In [15]:
# =====================================================
# define U-Net model architecture - proof_concept_2

def build_unet(img_shape):
    # input layer shape is equal to patch image size
    inputs = layers.Input(shape=img_shape)

    # rescale images from (0, 255) to (0, 1)
 #   rescale = Rescaling(scale=1. / 255, input_shape=(img_height, img_width, img_channels))(inputs)
 #   previous_block_activation = rescale  # Set aside residual
    previous_block_activation = inputs

    contraction = {}
    # # Contraction path: Blocks 1 through 5 are identical apart from the feature depth
    for f in [32, 64]:
        x = layers.Conv2D(f, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(previous_block_activation)
        x = layers.Dropout(0.1)(x)
        x = layers.Conv2D(f, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
        contraction[f'conv{f}'] = x
        x = layers.MaxPooling2D((2, 2))(x)
        previous_block_activation = x

    x = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(previous_block_activation)
    x = layers.Dropout(0.1)(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
    contraction[f'conv{128}'] = x
    x = layers.MaxPooling2D((3, 3))(x)
    previous_block_activation = x
        
    c5 = layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(previous_block_activation)
    c5 = layers.Dropout(0.2)(c5)
    c5 = layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)
    previous_block_activation = c5
    
    x = layers.Conv2DTranspose(128, (2, 2), strides=(3, 3), padding='same')(previous_block_activation)
    x = layers.concatenate([x, contraction[f'conv{128}']])
    x = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
    previous_block_activation = x
        
    # Expansive path: Second half of the network: upsampling inputs
    # could we use upsampling layers here instead of Conv2dTRanspose layers? might that help
    for f in reversed([32, 64]):
        x = layers.Conv2DTranspose(f, (2, 2), strides=(2, 2), padding='same')(previous_block_activation)
        x = layers.concatenate([x, contraction[f'conv{f}']])
        x = layers.Conv2D(f, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
        x = layers.Dropout(0.2)(x)
        x = layers.Conv2D(f, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
        previous_block_activation = x

    outputs = layers.Conv2D(filters=1, kernel_size=(1, 1), activation="linear")(previous_block_activation)

    return Model(inputs=inputs, outputs=outputs)

In [16]:
# build model
model_unet_canopy_ht_all = build_unet(img_shape=(240, 240, 3))
model_unet_canopy_ht_all.summary()

# compile model
model_unet_canopy_ht_all.compile(optimizer="adam",
              loss="mse", 
              metrics=["mae", 'accuracy'])

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 240, 240, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_41 (Conv2D)             (None, 240, 240, 32  896         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 dropout_20 (Dropout)           (None, 240, 240, 32  0           ['conv2d_41[0][0]']              
                                )                                                           

                                )                                                                 
                                                                                                  
 conv2d_55 (Conv2D)             (None, 240, 240, 1)  33          ['conv2d_54[0][0]']              
                                                                                                  
Total params: 1,925,601
Trainable params: 1,925,601
Non-trainable params: 0
__________________________________________________________________________________________________


In [17]:
%%time

callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

# Train our model
history=model_unet_canopy_ht_all.fit(
    training_generator,
    epochs=100,
    validation_data=val_generator,
    callbacks=callback
)

# model_unet_canopy_ht_all.save('C:/Users/johnf/Documents/UCL/thesis/code/models/height_model')

# hist_df = pd.DataFrame(history.history)

# hist_df
# hist_df.to_csv('C:/Users/johnf/Documents/UCL/thesis/code/models/tree_height_hist.csv')  

Epoch 1/100
478/478 [==============================] - 927s 2s/step - loss: 0.0133 - mae: 0.0831 - accuracy: 0.0765 - val_loss: 0.0116 - val_mae: 0.0795 - val_accuracy: 0.0768
Epoch 2/100
478/478 [==============================] - 939s 2s/step - loss: 0.0128 - mae: 0.0819 - accuracy: 0.0765 - val_loss: 0.0117 - val_mae: 0.0778 - val_accuracy: 0.0768
Epoch 3/100
478/478 [==============================] - 932s 2s/step - loss: 0.0123 - mae: 0.0787 - accuracy: 0.0765 - val_loss: 0.0114 - val_mae: 0.0805 - val_accuracy: 0.0768
Epoch 4/100
478/478 [==============================] - 937s 2s/step - loss: 0.0115 - mae: 0.0754 - accuracy: 0.0765 - val_loss: 0.0110 - val_mae: 0.0726 - val_accuracy: 0.0768
Epoch 5/100
478/478 [==============================] - 941s 2s/step - loss: 0.0117 - mae: 0.0746 - accuracy: 0.0765 - val_loss: 0.0118 - val_mae: 0.0796 - val_accuracy: 0.0768
Epoch 6/100
478/478 [==============================] - 948s 2s/step - loss: 0.0107 - mae: 0.0712 - accuracy: 0.0765 - va

KeyboardInterrupt: 

In [6]:
# vegetation mask ###


class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, batch_size=25, shuffle=True):
        'Initialization'
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)


    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        images = []

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            dataset = gdal.Open(ID)
            image = dataset.ReadAsArray()  # Returned image is a NumPy array with shape (16, 60, 60) for example.
            images.append(image)  # Append the NumPy array to the list.

        all_data= np.stack(images, axis= 0)
        all_data[all_data < .0000001] = 0
        X=all_data[:,:14,:,:] # separate out the band values
        X = np.transpose(X, axes=[0, 2, 3, 1])
        # normalize values of the input data to 0,1
        X = X/X.max(axis=(3),keepdims=1)
        # For RGB uncomment this
        X = X[:,:,:,:3]
        
        # canopy_height,tree/not tree,ndvi
        all_data= np.stack( images, axis= 0)
        Y = all_data[:,14:]
        Y[(Y < .0000001) & (Y >= 0) ] = 1
        #Y[:,0][Y[:,0]  > 1] = 1
        Y=Y[:,2] # 0 for height, 1 for tree/not, 2 for NDVI 
        Y[Y  >0 ] = 0
        Y[Y  <0 ] = 1
        #Y = Y/Y.max()
        self.X=X
        self.Y=Y

        return X, Y
    
    def get_true_values_x(self, indexes):
        'Generate one batch of data'
        # Return validation data for plotting 

        # Find list of IDs - (give input of validation indexes)
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)
        
        # only need to return the RGB data for plotting
        X = X[:,:,:,0:3]

        return X
    
    def get_true_values_y(self, indexes):
        'Generate one batch of data'
        # Return validation data for plotting 

        # Find list of IDs - (give input of validation indexes)
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return y

In [7]:
params = {'batch_size':15,
         'shuffle': True}

# need a dictionary which has a list of training paths and a list of validations paths
filelist_temp = filelist
np.random.seed(14)
mask = np.random.rand(len(filelist_temp)) <=.75

training_data = np.array(filelist_temp)[mask]
val_data = np.array(filelist_temp)[~mask]


mydict = {}
mydict["training"] = training_data
mydict["validation"] = val_data

# generators
training_generator = DataGenerator(mydict["training"], **params)
val_generator = DataGenerator(mydict["validation"], **params)

In [8]:
# =====================================================
# define U-Net model architecture - proof_concept_2

def build_unet(img_shape):
    # input layer shape is equal to patch image size
    inputs = layers.Input(shape=img_shape)

    # rescale images from (0, 255) to (0, 1)
 #   rescale = Rescaling(scale=1. / 255, input_shape=(img_height, img_width, img_channels))(inputs)
 #   previous_block_activation = rescale  # Set aside residual
    previous_block_activation = inputs

    contraction = {}
    # # Contraction path: Blocks 1 through 5 are identical apart from the feature depth
    for f in [32, 64]:
        x = layers.Conv2D(f, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(previous_block_activation)
        x = layers.Dropout(0.1)(x)
        x = layers.Conv2D(f, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
        contraction[f'conv{f}'] = x
        x = layers.MaxPooling2D((2, 2))(x)
        previous_block_activation = x

    x = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(previous_block_activation)
    x = layers.Dropout(0.1)(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
    contraction[f'conv{128}'] = x
    x = layers.MaxPooling2D((3, 3))(x)
    previous_block_activation = x
        
    c5 = layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(previous_block_activation)
    c5 = layers.Dropout(0.2)(c5)
    c5 = layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)
    previous_block_activation = c5
    
    x = layers.Conv2DTranspose(128, (2, 2), strides=(3, 3), padding='same')(previous_block_activation)
    x = layers.concatenate([x, contraction[f'conv{128}']])
    x = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
    previous_block_activation = x
        
    # Expansive path: Second half of the network: upsampling inputs
    # could we use upsampling layers here instead of Conv2dTRanspose layers? might that help
    for f in reversed([32, 64]):
        x = layers.Conv2DTranspose(f, (2, 2), strides=(2, 2), padding='same')(previous_block_activation)
        x = layers.concatenate([x, contraction[f'conv{f}']])
        x = layers.Conv2D(f, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
        x = layers.Dropout(0.2)(x)
        x = layers.Conv2D(f, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
        previous_block_activation = x

    outputs = layers.Conv2D(filters=1, kernel_size=(1, 1), activation="sigmoid")(previous_block_activation)

    return Model(inputs=inputs, outputs=outputs)

In [9]:
%%time

# build model
model_unet_ndvi_ht_all = build_unet(img_shape=(240, 240, 3))
model_unet_ndvi_ht_all.summary()

# compile model


# model_unet_ndvi_ht_all.compile(optimizer="adam",
#               loss="mse", 
#               metrics=["mae", 'accuracy'])
model_unet_ndvi_ht_all.compile(optimizer="adam",
              loss="binary_crossentropy", 
              metrics=[tf.keras.metrics.BinaryCrossentropy(),'accuracy',tf.keras.metrics.MeanIoU(num_classes=2)])



callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

# Train our model
history=model_unet_ndvi_ht_all.fit(
    training_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=callback
)

# model_unet_ndvi_ht_all.save('C:/Users/johnf/Documents/UCL/thesis/code/models/vegetation_model')

# hist_df = pd.DataFrame(history.history)

# hist_df
# hist_df.to_csv('C:/Users/johnf/Documents/UCL/thesis/code/models/vegetation_hist.csv')  

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 240, 240, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 240, 240, 32  896         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 dropout (Dropout)              (None, 240, 240, 32  0           ['conv2d[0][0]']                 
                                )                                                             

                                )                                                                 
                                                                                                  
 conv2d_14 (Conv2D)             (None, 240, 240, 1)  33          ['conv2d_13[0][0]']              
                                                                                                  
Total params: 1,925,601
Trainable params: 1,925,601
Non-trainable params: 0
__________________________________________________________________________________________________
Epoch 1/20
478/478 [==============================] - 1577s 3s/step - loss: 0.2281 - binary_crossentropy: 0.2281 - accuracy: 0.9036 - mean_io_u: 0.1256 - val_loss: 0.1669 - val_binary_crossentropy: 0.1669 - val_accuracy: 0.9345 - val_mean_io_u: 0.1264
Epoch 2/20
478/478 [==============================] - 931s 2s/step - loss: 0.1582 - binary_crossentropy: 0.1582 - accuracy: 0.9366 - mean_io_u: 0.1253 - val_loss: 0.1413 - val_b